In [1]:
from datasets import load_dataset
from transformers import AutoProcessor, AutoModelForCausalLM
import torch
from evaluate import load

torch.cuda.empty_cache()

dataset = load_dataset('eduvedras/VQG-Balanced',split='test',trust_remote_code=True)


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.0) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/eduvedras/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("eduvedras/git-base-vqg-unbalanced").to(device)
model

GitForCausalLM(
  (git): GitModel(
    (embeddings): GitEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(1024, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (image_encoder): GitVisionModel(
      (vision_model): GitVisionTransformer(
        (embeddings): GitVisionEmbeddings(
          (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
          (position_embedding): Embedding(197, 768)
        )
        (pre_layrnorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (encoder): GitVisionEncoder(
          (layers): ModuleList(
            (0-11): 12 x GitVisionEncoderLayer(
              (self_attn): GitVisionAttention(
                (k_proj): Linear(in_features=768, out_features=768, bias=True)
                (v_proj): Linear(in_features=768, out_features=768, bias=True)
             

In [3]:
checkpoint = "microsoft/git-base"
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained(checkpoint)

predictions = []
references = []

dic = {}

from tqdm.auto import tqdm

for i in range(len(dataset)):
    dic[dataset[i]['Chart_name']] = []

for i in range(len(dataset)):
    dic[dataset[i]['Chart_name']].append(dataset[i]['Question'])
    
print(len(dic))

16


In [4]:
for i in tqdm(range(len(dataset))):
    inputs = processor(images=dataset[i]['Chart'], return_tensors="pt").to(device)
    pixel_values = inputs.pixel_values
    generated_ids = model.generate(pixel_values=pixel_values, max_length=70)
    generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    predictions.append(generated_caption)
    references.append(dic[dataset[i]['Chart_name']])

100%|██████████| 826/826 [10:17<00:00,  1.34it/s]


In [5]:
import evaluate

filetag = "unbalanced"

file = open(f"predictions-{filetag}.txt", "a")

bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=references)
print(results)
file.write(f"BLEU: {results}\n")

meteor = evaluate.load("meteor")
results = meteor.compute(predictions=predictions, references=references)
print(results)
file.write(f"METEOR: {results}\n")

rouge = evaluate.load("rouge")
results = rouge.compute(predictions=predictions, references=references)
print(results)
file.write(f"ROUGE: {results}\n")
file.close()

{'bleu': 0.6170608737975198, 'precisions': [0.8043017868960953, 0.645267432091851, 0.5715559518502006, 0.4887587080430652], 'brevity_penalty': 1.0, 'length_ratio': 1.9187301587301586, 'translation_length': 15110, 'reference_length': 7875}


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/eduvedras/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/eduvedras/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/eduvedras/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


{'meteor': 0.8810379432474729}
{'rouge1': 0.878058067316942, 'rouge2': 0.8291339455659011, 'rougeL': 0.8778256930968913, 'rougeLsum': 0.8775443693862253}


In [14]:
import pandas as pd

new_df = pd.DataFrame(columns=['Image','Prediction'])

i=0
while i < len(dataset):
    if ((new_df['Image'] == dataset[i]['Chart_name']) & (new_df['Prediction'] == predictions[i])).any():
        i += 1
        continue
    else:
        new_df.loc[len(new_df)] = {'Image': dataset[i]['Chart_name'], 'Prediction': predictions[i]}
        i += 1
    
new_df.to_csv(f'predictions-{filetag}.csv', index=False)